# Analysis_5
- find the pokemon 'nest' in New York according to the coordinates 

In [4]:
import requests
import time
import json
import pandas as pd
from pandas import Series, DataFrame
import os

In [22]:
# Dataset consists of roughly 293,000 historical appearances of Pokemon
GO_data = pd.read_csv('../data/300k.csv')
# filter cols
GO_data = GO_data.loc[:,['pokemonId','latitude','longitude','appearedLocalTime','appearedTimeOfDay','city','continent','weather','population_density','urban','suburban','midurban']] 
GO_data.head()

/home/pandakillalot/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,pokemonId,latitude,longitude,appearedLocalTime,appearedTimeOfDay,city,continent,weather,population_density,urban,suburban,midurban
0,16,20.525745,-97.460829,2016-09-08T03:57:45,night,Mexico_City,America,Foggy,2431.2341,True,True,True
1,133,20.523695,-97.461167,2016-09-08T03:57:37,night,Mexico_City,America,Foggy,2431.2341,True,True,True
2,16,38.903590,-77.199780,2016-09-08T03:57:25,night,New_York,America,Clear,761.8856,False,True,True
3,13,47.665903,-122.312561,2016-09-08T03:56:22,night,Los_Angeles,America,PartlyCloudy,4842.1626,True,True,True
4,133,47.666454,-122.311628,2016-09-08T03:56:08,night,Los_Angeles,America,PartlyCloudy,4842.1626,True,True,True


In [23]:
# all data size
GO_data.shape

(296021, 12)

In [24]:
# 'New_York' data size 
GO_data = GO_data[GO_data['continent'] == 'America']
GO_data.shape

(150545, 12)

In [25]:
GO_data.head()

,pokemonId,latitude,longitude,appearedLocalTime,appearedTimeOfDay,city,continent,weather,population_density,urban,suburban,midurban
0,16,20.525745,-97.460829,2016-09-08T03:57:45,night,Mexico_City,America,Foggy,2431.2341,True,True,True
1,133,20.523695,-97.461167,2016-09-08T03:57:37,night,Mexico_City,America,Foggy,2431.2341,True,True,True
2,16,38.903590,-77.199780,2016-09-08T03:57:25,night,New_York,America,Clear,761.8856,False,True,True
3,13,47.665903,-122.312561,2016-09-08T03:56:22,night,Los_Angeles,America,PartlyCloudy,4842.1626,True,True,True
4,133,47.666454,-122.311628,2016-09-08T03:56:08,night,Los_Angeles,America,PartlyCloudy,4842.1626,True,True,True


# Use google map api to turn coordinate into real location

In [26]:
# use google map to know the  location that the coordinate stand for

In [7]:
url = 'https://maps.googleapis.com/maps/api/geocode/json?latlng='

In [8]:
# get key from envirn
key = '&key=' + os.environ.get('google_key')

In [9]:
#GO_data['location_geo'] = str(GO_data['latitude']) +","+ str(GO_data['longitude'])
#GO_data.head()

In [15]:
# get lat&lon coordinate from the row and apply it to the search url
def my_test2(row):
    urrl = url + str(row['latitude']) +","+str(row['longitude']) + key
    aaa = requests.get(urrl)
    j = aaa.json()
    return j['results'][0]['address_components'][3]['long_name'] + "，" + j['results'][0]['address_components'][4]['long_name']

In [16]:
# def to do an 'for (int i = 0; i < 100; i++) like loop'
def my_range(start, end, step):
    while start <= end:
        yield start
        start += step

In [17]:
# divde the dataframe in to small dataframe
# get the neighbour info from the response json and write
# write the answer  to the csv file
def copy_transfer(df,start,end,step,name):
    turn = name
    for x in my_range(start, end, step):
        df_copy = df[x - 100:x].copy()
        df_copy['neigbour'] = df_copy.apply(my_test2,axis = 1)
        name ='ana_3_output/GO_data_'+ str(turn)+ '.csv' 
        df_copy.to_csv(name, encoding='utf-8')
        turn = turn + 1
        time.sleep(50)

In [19]:
copy_transfer(GO_data,2400,3200,100,22)

# Find the most popular neighbourhood in United Stats

In [9]:
l_csv = os.listdir('../data/data_google_map')

In [10]:
len(l_csv)

30

In [15]:
def read_all_neigbour(list_file_name):
    l_all =[]
    for x in list_file_name:
        csv_1 = pd.read_csv('../data/data_google_map/' + x)
        s = list(csv_1['neigbour'].values)
        l_all = l_all + s
    return l_all

In [16]:
l_nei = read_all_neigbour(os.listdir('../data/data_google_map'))
len(l_nei)

3000

In [17]:
l_county = [x.split('，')[1] for x in l_nei]

In [18]:
from nltk import FreqDist

In [37]:
def count_most(lists):
    fd = FreqDist(lists) 
    word_list = []
    for word in fd.keys():
        word_list.append((word, fd[word])) 
    sorted_list = sorted(word_list, key=lambda t:t[1], reverse=True)
    print('Most Frequent Words in Persuasion:\n', sorted_list[:10])
    return sorted_list

In [39]:
hot_county = count_most(l_county)

Most Frequent Words in Persuasion:
 [('Poza Rica', 202), ('Texas', 180), ('Georgia', 132), ('Chittenden County', 129), ('DuPage County', 117), ('Division No. 11', 106), ('United States', 95), ('Frederick County', 95), ('California', 93), ('Toronto', 78)]


In [40]:
l_neigbour = [x.split('，')[0] for x in l_nei if x.split('，')[1]]

In [41]:
def find_neighbour(county):
    l_neigbour = [x.split('，')[0] for x in l_nei if x.split('，')[1] == county]
    return l_neigbour

In [42]:
l = find_neighbour('Texas')

In [43]:
hot_neighbour = count_most(l)

Most Frequent Words in Persuasion:
 [('Hidalgo County', 55), ('Tarrant County', 42), ('Denton County', 37), ('Galveston County', 27), ('Cameron County', 11), ('Fort Bend County', 8)]


In [45]:
dic_rank = {}
rank = 1
for x in hot_county:
    county = x[0]
    number = x[1]
    dic_rank[county] = {'rank':rank,'number':x[1],'hot_neighborhood': count_most(find_neighbour(county))}
    rank = rank + 1

Most Frequent Words in Persuasion:
 [('Poza Rica', 202)]
Most Frequent Words in Persuasion:
 [('Hidalgo County', 55), ('Tarrant County', 42), ('Denton County', 37), ('Galveston County', 27), ('Cameron County', 11), ('Fort Bend County', 8)]
Most Frequent Words in Persuasion:
 [('Clayton County', 79), ('Upson County', 32), ('Fulton County', 21)]
Most Frequent Words in Persuasion:
 [('Burlington', 129)]
Most Frequent Words in Persuasion:
 [('Naperville Township', 117)]
Most Frequent Words in Persuasion:
 [('Edmonton', 105), ('Winnipeg', 1)]
Most Frequent Words in Persuasion:
 [('California', 33), ('Texas', 15), ('Florida', 11), ('Washington', 8), ('Georgia', 8), ('Massachusetts', 7), ('Virginia', 4), ('Colorado', 3), ('South Carolina', 3), ('Vermont', 1)]
Most Frequent Words in Persuasion:
 [('2, Frederick', 95)]
Most Frequent Words in Persuasion:
 [('Los Angeles County', 44), ('Merced County', 20), ('San Mateo County', 12), ('San Diego County', 11), ('Sonoma County', 5), ('San Francisco 

AttributeError: 'dict' object has no attribute 'size'

In [46]:
with open('ana_5_hot.json','w')as f:
    json.dump(dic_rank, f)